# Kütüphaneler

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector,Bidirectional
#from keras.layers import Input, Dense, RepeatVector, , Dropout, LSTM
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import load_model

# Veri setini yükle
source_text: ingilizce

target_text: almanca

## txt to csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_text(filename):
        # open the file
        file = open(filename, mode='rt', encoding='utf-8')

        # read all text
        text = file.read()
        file.close()
        return text

def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents

data = read_text(path)
deu_eng = to_lines(data)
deu_eng = array(deu_eng)

In [ ]:
eng=[deu_eng[x][0] for x in range(len(deu_eng))]
deu=[deu_eng[x][1] for x in range(len(deu_eng))]
print(eng[0])
print(deu[0])

In [ ]:
df = {'eng': eng,
    'deu': deu}

df = pd.DataFrame(df, columns=['eng','deu'])
df.to_csv(filepath)

## read_csv

In [ ]:
filepath = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu_eng.csv'
df=pd.read_csv(filepath)
df.head(2)

,Unnamed: 0,eng,deu
0,0,Go.,Geh.
1,1,Hi.,Hallo!


In [ ]:
df.columns

Index(['Unnamed: 0', 'eng', 'deu'], dtype='object')

In [ ]:
len(df)

227080

In [ ]:
df=df[['eng', 'deu']]
df.head(5)

,eng,deu
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!


In [ ]:
len(df)

227080

In [ ]:
df.head(5)

,eng,deu
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!


In [ ]:
df.isna().sum().sum()

0

In [ ]:
# df=df[:5000]
# en_sentences=df['eng']
# de_sentences=df['deu']

# print("english: ",en_sentences[0],"\ngerman: ",de_sentences[0])

english:  Go. 
german:  Geh.


## train test split

In [ ]:
from sklearn.model_selection import train_test_split
df=df[:5000]
X , y = df['eng'], df['deu']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [ ]:
en_sentences=X_train
de_sentences=y_train
print("english: ",en_sentences[0],"\ngerman: ",de_sentences[0])

english:  Go. 
german:  Geh.


## Save training set as csv

In [ ]:
for x in X_train:
  print(x)

I'm eating.
I'm crazy.
Get down!


In [ ]:
a=X_train[:3]
print(a)
index_ = [0,1,2]
a.index = index_
print(a)

1522    I'm eating.
835      I'm crazy.
358       Get down!
Name: eng, dtype: object
0    I'm eating.
1     I'm crazy.
2      Get down!
Name: eng, dtype: object


In [ ]:
len(X_train)

3350

In [ ]:
index_=[x for x in range(0,len(X_train))]
X_train.index = index_
y_train.index = index_
df2 = {'eng': X_train,
    'deu': y_train}
df2 = pd.DataFrame(df2, columns=['eng','deu'])
df2.shape
filepath = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu_eng_train.csv'
df2.to_csv(filepath)

## Save testing set as csv

In [ ]:
len(X_test)

1650

In [ ]:
index_=[x for x in range(0,len(X_test))]
X_test.index = index_
y_test.index = index_
df2 = {'eng': X_test,
    'deu': y_test}
df2 = pd.DataFrame(df2, columns=['eng','deu'])
df2.shape
filepath = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu_eng_test.csv'
df2.to_csv(filepath)

# Preprocessing


## Load train&test data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#df_train
train_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu_eng_train.csv'
df_train=pd.read_csv(train_path)
df_train.head(2)

X_train, y_train= df_train['eng'],df_train['deu']
en_sentences=df_train['eng']
de_sentences=df_train['deu']
print("english: ",en_sentences[5],"\ngerman: ",de_sentences[5])


english:  I've eaten. 
german:  Ich habe gegessen.


In [7]:
#df_test
test_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/deu_eng_test.csv'
df_test=pd.read_csv(test_path)
print(df_test.head(2))

X_test, y_test= df_test['eng'],df_test['deu']

   Unnamed: 0           eng                      deu
0           0   I'll hurry.  Ich werde mich beeilen.
1           1  Is it tasty?              Schmeckt's?


## Tokenization

In [8]:
#en_sentences tokenization
en_tokenizer = Tokenizer()
en_tokenizer.fit_on_texts(en_sentences)
en_sequences=en_tokenizer.texts_to_sequences(en_sentences)
en_indexes=en_tokenizer.word_index # returns english vocabulary
print("en_indexes: ",en_indexes)
print("en_vocab_size: ",len(en_indexes))
print("en_sequences[]: ",en_sequences)


print("\n")
#de_sentences tokenization
de_tokenizer = Tokenizer()
de_tokenizer.fit_on_texts(de_sentences)
de_sequences=de_tokenizer.texts_to_sequences(de_sentences)
de_indexes=de_tokenizer.word_index# returns german vocabulary
print("de_indexes: ",de_indexes)
print("de_vocab_size: ",len(de_indexes))
print("de_sequences: ",de_sequences)

en_indexes:  {'tom': 1, 'i': 2, 'it': 3, "i'm": 4, 'is': 5, 'you': 6, 'a': 7, 'me': 8, 'we': 9, 'go': 10, "it's": 11, 'he': 12, 'come': 13, 'was': 14, 'this': 15, 'get': 16, 'can': 17, 'that': 18, 'who': 19, "don't": 20, 'are': 21, 'be': 22, "tom's": 23, 'up': 24, 'away': 25, 'they': 26, 'in': 27, "i'll": 28, 'on': 29, 'do': 30, 'keep': 31, "we're": 32, 'she': 33, 'take': 34, 'lost': 35, 'here': 36, 'stop': 37, 'let': 38, 'did': 39, 'am': 40, 'try': 41, 'no': 42, 'us': 43, 'help': 44, "that's": 45, "you're": 46, 'need': 47, 'love': 48, 'have': 49, 'out': 50, 'home': 51, "let's": 52, 'like': 53, 'saw': 54, 'look': 55, 'down': 56, 'now': 57, 'not': 58, 'please': 59, "he's": 60, 'got': 61, 'how': 62, 'see': 63, 'him': 64, 'them': 65, 'for': 66, 'know': 67, 'ok': 68, 'to': 69, 'wait': 70, 'eat': 71, "we'll": 72, 'may': 73, 'want': 74, 'there': 75, 'busy': 76, 'back': 77, 'my': 78, 'stay': 79, "they're": 80, 'hurry': 81, "who's": 82, 'what': 83, 'one': 84, 'will': 85, 'hurt': 86, 'turn': 87

In [9]:
print(en_sentences[0])
print(en_sequences[0])

I'm eating.
[4, 283]


In [10]:
key = list(filter(lambda x: en_indexes[x] == 283, en_indexes))[0]
print(key)

eating


## Padding

In [11]:
en_preprocessed=pad_sequences(en_sequences, maxlen=None, padding='post')#maxlen: If None, use length of longest sequence in en_sequences.
de_preprocessed=pad_sequences(de_sequences, maxlen=None, padding='post')

In [12]:
print(de_preprocessed.shape)
print(en_preprocessed.shape)

(3350, 8)
(3350, 4)


In [13]:
max_en_length=len(en_preprocessed[0])
max_de_length=len(de_preprocessed[0])
print("maximum english sequence length: ",max_en_length)
print("maximum german sequence length: ",max_de_length)

print("\nPadded examples of the sequences:\n")
print(en_preprocessed[0])
print(de_preprocessed[0])

maximum english sequence length:  4
maximum german sequence length:  8

Padded examples of the sequences:

[  4 283   0   0]
[  1 114   0   0   0   0   0   0]


In [14]:
en_preprocessed=pad_sequences(en_sequences, maxlen=max_de_length, padding='post')
print("\nPadded examples of the sequences:\n")
print(en_preprocessed[0]," **lenght=",len(en_preprocessed[0]),"**")
print(de_preprocessed[0]," **lenght=",len(de_preprocessed[0]),"**")
max_de_length=len(de_preprocessed[0])


Padded examples of the sequences:

[  4 283   0   0   0   0   0   0]  **lenght= 8 **
[  1 114   0   0   0   0   0   0]  **lenght= 8 **


## functions

In [15]:
#convert prediction to text
def pred_to_text(pred):
  indexdict = {id: word for word, id in de_tokenizer.word_index.items()}
  indexdict[0] = '<PAD>'
  words=[]
  #print(np.argmax(pred, 1))
  for item in np.argmax(pred, 1):
    words.append(indexdict[item])
  words=[w for w in words if w!='<PAD>']
 # print(' '.join(words))
  return np.argmax(pred, 1),' '.join(words)

In [16]:
#convert test text into indexes form
def indexed_test_text(text):
  sequence = de_tokenizer.texts_to_sequences([text])
  new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
  return new_test[0]

In [17]:
#error function
def pred_error(indexed_text, indexed_pred):
  miss=0
  for i in range(len(indexed_text)):
    if indexed_text[i]!=indexed_pred[i]:
      miss=miss+1
  return miss/len(indexed_text)

# Model

##Model1 LSTM

### train model1(optimizer='adam)

In [35]:

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split( en_preprocessed, de_preprocessed, test_size=0.33, random_state=42)

input_shape=en_preprocessed.shape
output_shape=de_preprocessed.shape

de_vocab_size=len(de_indexes)
len(de_indexes)

print(input_shape)
print(output_shape)
print(de_vocab_size)

(3350, 8)
(3350, 8)
1835


In [51]:
model = Sequential()
model.add(Embedding(len(en_indexes)+1, 128, input_length=input_shape[1]))
#encoder
model.add(LSTM(128))
model.add(RepeatVector(output_shape[1]))
#decoder
model.add(LSTM(128,input_shape=input_shape[1:], return_sequences=True))

model.add(Dense(de_vocab_size+1, activation='softmax'))

# Compile model
model.compile(loss=sparse_categorical_crossentropy,optimizer="adam",metrics=['accuracy'])
print(model.summary())
model.fit(en_preprocessed, de_preprocessed, batch_size=32, epochs=150, validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8, 128)            147840    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 repeat_vector_1 (RepeatVec  (None, 8, 128)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 8, 128)            131584    
                                                                 
 dense_1 (Dense)             (None, 8, 1836)           236844    
                                                                 
Total params: 647852 (2.47 MB)
Trainable params: 647852 (2.47 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [52]:
#save model1
model1_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model1.joblib'
model.save(model1_path)

### train model3(optimizer='sgd')

In [36]:
model = Sequential()
model.add(Embedding(len(en_indexes)+1, 128, input_length=input_shape[1]))
#encoder
model.add(LSTM(128))
model.add(RepeatVector(output_shape[1]))
#decoder
model.add(LSTM(128,input_shape=input_shape[1:], return_sequences=True))

model.add(Dense(de_vocab_size+1, activation='softmax'))

# Compile model
model.compile(loss=sparse_categorical_crossentropy,optimizer="sgd",metrics=['accuracy'])
print(model.summary())
model.fit(en_preprocessed, de_preprocessed, batch_size=32, epochs=150, validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 128)            147840    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 repeat_vector (RepeatVecto  (None, 8, 128)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 8, 128)            131584    
                                                                 
 dense (Dense)               (None, 8, 1836)           236844    
                                                                 
Total params: 647852 (2.47 MB)
Trainable params: 647852 (2.47 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [37]:
#save model
model3_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model3.joblib'
model.save(model3_path)

### test model1

In [53]:
#load model
import tensorflow as tf
model1_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model1.joblib'
model1 = tf.keras.models.load_model(model1_path)

In [54]:
print(model1.predict(en_preprocessed[:1])[0])
print(pred_to_text(model1.predict(en_preprocessed[:1])[0]))

1/1 [==============================] - 1s 1s/step
[[1.0852938e-08 9.9976254e-01 8.4954260e-10 ... 3.2414466e-13
  4.9113502e-13 5.4215410e-13]
 [1.5045307e-06 6.4415124e-04 9.5404999e-08 ... 2.9393968e-10
  5.8309896e-10 4.6616205e-10]
 [1.7563278e-02 6.7442452e-04 1.7919474e-06 ... 4.1468531e-09
  1.0457969e-08 7.4817619e-09]
 ...
 [9.7581780e-01 1.0092890e-08 1.2025763e-09 ... 6.7425796e-12
  1.0620301e-11 1.0916475e-11]
 [9.9766308e-01 2.1120622e-10 2.1711573e-10 ... 6.4155967e-13
  1.0925168e-12 9.9281824e-13]
 [9.9949270e-01 7.3829290e-11 6.7063660e-11 ... 1.6117870e-13
  2.9832200e-13 2.5903295e-13]]
1/1 [==============================] - 0s 38ms/step
(array([  1, 114, 188,   0,   0,   0,   0,   0]), 'ich esse gerade')


predict single text

In [55]:
text="I'm sorry."
sequence = en_tokenizer.texts_to_sequences([text])
new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
print(new_test[0])
print(len(new_test[0]))
pred = model1.predict(new_test)[0]
pred_to_text(pred)

[  4 220   0   0   0   0   0   0]
8
1/1 [==============================] - 0s 22ms/step


(array([  5,  92,  17, 162,   0,   0,   0,   0]), 'es tut mir leid')

predict X_test

In [60]:
y_test

0              Ich werde mich beeilen.
1                          Schmeckt's?
2                       Spring drüber.
3                            Nimm das.
4                        Hier bin ich.
                     ...              
1645                    Es ist unsers.
1646                       Kegelt ihr?
1647    Ich werde schon zurechtkommen.
1648               Lassen Sie Tom los!
1649                     Dies ist Tom.
Name: deu, Length: 1650, dtype: object

In [61]:
X_test

0         I'll hurry.
1        Is it tasty?
2        Jump across.
3          Take this.
4          Here I am.
            ...      
1645       It's ours.
1646     Do you bowl?
1647    I'll be fine.
1648      Let Tom go.
1649     This is Tom.
Name: eng, Length: 1650, dtype: object

In [56]:
test_sequence = en_tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(test_sequence,padding='post', maxlen=max_de_length)
predictions=model1.predict(padded_test)

52/52 [==============================] - 1s 15ms/step


In [57]:
predictions.shape

(1650, 8, 1836)

In [58]:
# indexed and text forms of the X_test predictions
pred_indexed=[]
pred_sentences=[]
for i in range(len(predictions)):
  pred_indexed.append(pred_to_text(predictions[i])[0])
  pred_sentences.append(pred_to_text(predictions[i])[1])
print(pred_indexed[:5])
print(pred_sentences[:5])

[array([  1,  27, 764,   0,   0,   0,   0,   0]), array([879,   5,   0,   0,   0,   0,   0,   0]), array([771, 772,   0,   0,   0,   0,   0,   0]), array([1202,    7,    0,    0,    0,    0,    0,    0]), array([  1,   1, 320,   0,   0,   0,   0,   0])]
['ich werde anrufen', 'schmeckt es', 'halt’s maul', 'unterschreibe das', 'ich ich gerecht']


In [59]:
data = {'X_test': X_test,
    'y_test': y_test,
        'predicitons': pred_sentences}
df = pd.DataFrame(data)
print(df)

             X_test                          y_test          predicitons
0       I'll hurry.         Ich werde mich beeilen.    ich werde anrufen
1      Is it tasty?                     Schmeckt's?          schmeckt es
2      Jump across.                  Spring drüber.          halt’s maul
3        Take this.                       Nimm das.    unterschreibe das
4        Here I am.                   Hier bin ich.      ich ich gerecht
...             ...                             ...                  ...
1645     It's ours.                  Es ist unsers.           das ist zu
1646   Do you bowl?                     Kegelt ihr?           kegeln sie
1647  I'll be fine.  Ich werde schon zurechtkommen.  ich komm schon klar
1648    Let Tom go.             Lassen Sie Tom los!       lass tom gehen
1649   This is Tom.                   Dies ist Tom.      das spricht tom

[1650 rows x 3 columns]


## hata payı

In [62]:
text="This is Tom."
sequence = en_tokenizer.texts_to_sequences([text])
new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
print(new_test[0])
print(len(new_test[0]))
pred = model1.predict(new_test)[0]
indexed_pred, text_pred = pred_to_text(pred)
print(indexed_pred)
print(text_pred)

[15  5  1  0  0  0  0  0]
8
1/1 [==============================] - 0s 23ms/step
[   7 1249    2    0    0    0    0    0]
das spricht tom


In [64]:
text="Dies ist Tom."
indexed_text=indexed_test_text(text)
indexed_text

array([321,   3,   2,   0,   0,   0,   0,   0], dtype=int32)

In [65]:
print(indexed_pred)
print(indexed_text)

[   7 1249    2    0    0    0    0    0]
[321   3   2   0   0   0   0   0]


In [72]:
pred_error(indexed_text, indexed_pred)

0.25

In [73]:
# find errors for each text in y_test
errors=[]
indexed_text=y_test.apply(indexed_test_text)
for i in range(len(pred_indexed)):
  errors.append(pred_error(indexed_text[i], pred_indexed[i]))


In [74]:
data = {'X_test': X_test,
    'y_test': y_test,
        'predicitons': pred_sentences,
        'error': errors}
df = pd.DataFrame(data)
print(df)

             X_test                          y_test          predicitons  \
0       I'll hurry.         Ich werde mich beeilen.    ich werde anrufen   
1      Is it tasty?                     Schmeckt's?          schmeckt es   
2      Jump across.                  Spring drüber.          halt’s maul   
3        Take this.                       Nimm das.    unterschreibe das   
4        Here I am.                   Hier bin ich.      ich ich gerecht   
...             ...                             ...                  ...   
1645     It's ours.                  Es ist unsers.           das ist zu   
1646   Do you bowl?                     Kegelt ihr?           kegeln sie   
1647  I'll be fine.  Ich werde schon zurechtkommen.  ich komm schon klar   
1648    Let Tom go.             Lassen Sie Tom los!       lass tom gehen   
1649   This is Tom.                   Dies ist Tom.      das spricht tom   

      error  
0     0.250  
1     0.250  
2     0.250  
3     0.125  
4     0.375  
...

In [75]:
print("LSTM Model error: ",sum(errors)/len(pred_indexed))

LSTM Model error:  0.2484090909090909


# Model2 lstm+bidirectional

## train model2(optimizer='adam')

In [76]:

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split( en_preprocessed, de_preprocessed, test_size=0.33, random_state=42)

input_shape=en_preprocessed.shape
output_shape=de_preprocessed.shape

de_vocab_size=len(de_indexes)
len(de_indexes)

print(input_shape)
print(output_shape)
print(de_vocab_size)

(3350, 8)
(3350, 8)
1835


In [77]:
model = Sequential()
model.add(Embedding(len(en_indexes)+1, 128, input_length=input_shape[1]))#, mask_zero=True
#encoder
lstm=LSTM(128)
model.add(Bidirectional(lstm))
model.add(RepeatVector(output_shape[1]))

#decoder
model.add(Bidirectional(LSTM(128,input_shape=input_shape[1:], return_sequences=True)))

model.add(Dense(de_vocab_size+1, activation='softmax'))

# Compile model
model.compile(loss=sparse_categorical_crossentropy,optimizer="adam",metrics=['accuracy'])
print(model.summary())
model.fit(en_preprocessed, de_preprocessed, batch_size=32, epochs=150, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 8, 128)            147840    
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 repeat_vector_2 (RepeatVec  (None, 8, 256)            0         
 tor)                                                            
                                                                 
 bidirectional_1 (Bidirecti  (None, 8, 256)            394240    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 8, 1836)           471852    
                                                      

In [78]:
#save model2
model2_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model2.joblib'
model.save(model2_path)

### train model4(optimizer='sgd')

In [38]:
model = Sequential()
model.add(Embedding(len(en_indexes)+1, 128, input_length=input_shape[1]))
#encoder
lstm=LSTM(128)
model.add(Bidirectional(lstm))
model.add(RepeatVector(output_shape[1]))

#decoder
model.add(Bidirectional(LSTM(128,input_shape=input_shape[1:], return_sequences=True)))

model.add(Dense(de_vocab_size+1, activation='softmax'))

# Compile model
model.compile(loss=sparse_categorical_crossentropy,optimizer="sgd",metrics=['accuracy'])
print(model.summary())
model.fit(en_preprocessed, de_preprocessed, batch_size=32, epochs=150, validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8, 128)            147840    
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 repeat_vector_1 (RepeatVec  (None, 8, 256)            0         
 tor)                                                            
                                                                 
 bidirectional_1 (Bidirecti  (None, 8, 256)            394240    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 8, 1836)           471852    
                                                      

In [ ]:
#save model
model4_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model4.joblib'
model.save(model4_path)

## test model2

In [19]:
#load model
import tensorflow as tf
model2_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model2.joblib'
model2 = tf.keras.models.load_model(model2_path)

In [21]:
text="I'm sorry."
sequence = en_tokenizer.texts_to_sequences([text])
new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
print(new_test[0])
print(len(new_test[0]))
pred = model2.predict(new_test)[0]
pred_to_text(pred)

[  4 220   0   0   0   0   0   0]
8
1/1 [==============================] - 3s 3s/step


(array([  5,  92,  17, 162,   0,   0,   0,   0]), 'es tut mir leid')

In [22]:
test_sequence = en_tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(test_sequence,padding='post', maxlen=max_de_length)
predictions=model2.predict(padded_test)

52/52 [==============================] - 1s 25ms/step


In [23]:
# indexed and text forms of the X_test predictions
pred_indexed=[]
pred_sentences=[]
for i in range(len(predictions)):
  pred_indexed.append(pred_to_text(predictions[i])[0])
  pred_sentences.append(pred_to_text(predictions[i])[1])
print(pred_indexed[:5])
print(pred_sentences[:5])

[array([ 1, 27, 15,  0,  0,  0,  0,  0]), array([879,   5,   0,   0,   0,   0,   0,   0]), array([373, 262,   0,   0,   0,   0,   0,   0]), array([80,  4,  0,  0,  0,  0,  0,  0]), array([7, 3, 7, 0, 0, 0, 0, 0])]
['ich werde dich', 'schmeckt es', 'spring runter', 'nimm sie', 'das ist das']


In [24]:
data = {'X_test': X_test,
    'y_test': y_test,
        'predicitons': pred_sentences}
df = pd.DataFrame(data)
print(df)

             X_test                          y_test          predicitons
0       I'll hurry.         Ich werde mich beeilen.       ich werde dich
1      Is it tasty?                     Schmeckt's?          schmeckt es
2      Jump across.                  Spring drüber.        spring runter
3        Take this.                       Nimm das.             nimm sie
4        Here I am.                   Hier bin ich.          das ist das
...             ...                             ...                  ...
1645     It's ours.                  Es ist unsers.        es ist montag
1646   Do you bowl?                     Kegelt ihr?           kegelst du
1647  I'll be fine.  Ich werde schon zurechtkommen.  ich komm schon klar
1648    Let Tom go.             Lassen Sie Tom los!      lasst tom gehen
1649   This is Tom.                   Dies ist Tom.          das ist tom

[1650 rows x 3 columns]


## hata payı

In [25]:
text="This is Tom."
sequence = en_tokenizer.texts_to_sequences([text])
new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
print(new_test[0])
print(len(new_test[0]))
pred = model2.predict(new_test)[0]
indexed_pred, text_pred = pred_to_text(pred)
print(indexed_pred)
print(text_pred)

[15  5  1  0  0  0  0  0]
8
1/1 [==============================] - 0s 42ms/step
[7 3 2 0 0 0 0 0]
das ist tom


In [26]:
text="Dies ist Tom."
indexed_text=indexed_test_text(text)
indexed_text

array([321,   3,   2,   0,   0,   0,   0,   0], dtype=int32)

In [27]:
print(indexed_pred)
print(indexed_text)

[7 3 2 0 0 0 0 0]
[321   3   2   0   0   0   0   0]


In [28]:
pred_error(indexed_text, indexed_pred)

0.125

In [29]:
# find errors for each text in y_test
errors=[]
indexed_text=y_test.apply(indexed_test_text)
for i in range(len(pred_indexed)):
  errors.append(pred_error(indexed_text[i], pred_indexed[i]))


In [30]:
data = {'X_test': X_test,
    'y_test': y_test,
        'predicitons': pred_sentences,
        'error': errors}
df2 = pd.DataFrame(data)
print(df2)

             X_test                          y_test          predicitons  \
0       I'll hurry.         Ich werde mich beeilen.       ich werde dich   
1      Is it tasty?                     Schmeckt's?          schmeckt es   
2      Jump across.                  Spring drüber.        spring runter   
3        Take this.                       Nimm das.             nimm sie   
4        Here I am.                   Hier bin ich.          das ist das   
...             ...                             ...                  ...   
1645     It's ours.                  Es ist unsers.        es ist montag   
1646   Do you bowl?                     Kegelt ihr?           kegelst du   
1647  I'll be fine.  Ich werde schon zurechtkommen.  ich komm schon klar   
1648    Let Tom go.             Lassen Sie Tom los!      lasst tom gehen   
1649   This is Tom.                   Dies ist Tom.          das ist tom   

      error  
0     0.250  
1     0.250  
2     0.125  
3     0.125  
4     0.375  
...

In [31]:
print("Bidirectional model error: ",sum(errors)/len(pred_indexed))

Bidirectional model error:  0.24742424242424244


# Uygulama

In [ ]:
#load model
import tensorflow as tf
model_path = '/content/drive/My Drive/Essential_AI_Bootcamp/proje/model2.joblib'
model = tf.keras.models.load_model(model_path)

print("İNGİLİZCE - ALMANCA ÇEVİRİ UYGULMASI")
print("Not: Uygulamadan çıkmak için text alanına 0 girin.\n\n")
while (True):
  text=input("İngilizce text: ")
  if text == '0':
    print("Uygulamadan çıkılıyor!")
    break
  else:
    sequence = en_tokenizer.texts_to_sequences([text])
    new_test = pad_sequences(sequence,padding='post', maxlen=max_de_length)
    pred = model.predict(new_test)[0]
    indexed_pred, text_pred = pred_to_text(pred)
    print("Almanca text: ",text_pred)
    print("\n\n")

İNGİLİZCE - ALMANCA ÇEVİRİ UYGULMASI
Not: Uygulamadan çıkmak için text alanına 0 girin.


İngilizce text: Tom
1/1 [==============================] - 1s 1s/step
Almanca text:  tom hat sich



İngilizce text: 0
Uygulamadan çıkılıyor!
